In large-language-model (LLM) research, **faithfulness** (sometimes called *factual consistency* or *groundedness*) measures **how strictly an LLM’s output adheres to its stated sources, its own earlier reasoning steps, or the user-provided context**. In plainer words: *“Does the model stick to the facts and evidence it was given, without inventing or distorting them?”*

---

## Why faithfulness matters

| Where it crops up                        | Consequences of low faithfulness                                          |
| ---------------------------------------- | ------------------------------------------------------------------------- |
| **Retrieval-augmented answers**          | Users lose trust if the summary contradicts the very documents it cites.  |
| **Chain-of-thought reasoning**           | An elegant derivation is useless if later steps contradict earlier logic. |
| **Legal/medical writing**                | Even a single ungrounded statement can be harmful or inadmissible.        |
| **Data-to-text (e.g., charts → report)** | Decision makers may act on numbers the model never actually saw.          |

---

## Bottom line

Faithfulness is the **“truth-in-context” filter**: it doesn’t ask whether the world itself agrees, only whether the model’s output can be justified by the evidence at hand or by its own logical steps. High-faithfulness systems combine strong retrieval, clear citing instructions, constrained decoding, and verification loops so that every claim can be traced back to its source—vital for trustworthy deployment in any domain where facts really matter.


In [1]:
import os
import wikipedia
import tiktoken
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain.schema import SystemMessage, HumanMessage
from IPython.display import display, Markdown
from pathlib import Path
from dotenv import load_dotenv

from deepeval import evaluate
from deepeval.metrics import FaithfulnessMetric
from deepeval.test_case import LLMTestCase

In [2]:
dotenv_path = Path("../../.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
model_eval = ChatOpenAI(model="gpt-4o-mini")

In [4]:
enc = tiktoken.get_encoding("o200k_base")

## Recovering the context (from wikipedia)

In [5]:
wikipedia.set_lang("en")
page = wikipedia.page("Albert Einstein")
context = page.content
display(Markdown(f"**Context:**"))
display(Markdown(f"----"))
display(Markdown(f"{context[:500]}...{context[-500:]}"))
display(Markdown(f"----"))
display(Markdown(f"Estimated number of tokens: {len(enc.encode(context))}"))

**Context:**

----

Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is best known for developing the theory of relativity. Einstein also made important contributions to quantum mechanics. His mass–energy equivalence formula E = mc2, which arises from special relativity, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics for his services to theoretical physics, and especially for his discovery of the law of the photoelectric eff...in
Finding aid to Albert Einstein Collection from Center for Jewish History


==== Digital collections ====
The Digital Einstein Papers — An open-access site for The Collected Papers of Albert Einstein, from Princeton University
Albert Einstein Digital Collection from Vassar College Digital Collections
Newspaper clippings about Albert Einstein in the 20th Century Press Archives of the ZBW
Albert – The Digital Repository of the IAS, which contains many digitized original documents and photographs

----

Estimated number of tokens: 17617

## Question

In [6]:
chat = ChatOllama(model="llama3.2")

prompt = f"""
Who was Albert Einstein and what did he do?"""

response = chat.invoke(prompt)
answer = response.content

display(Markdown(f"**Prompt:**"))
display(Markdown(f"----"))
display(Markdown(prompt))
display(Markdown(f"----"))

display(Markdown(f"**Answer:**"))
display(Markdown(f"----"))
display(Markdown(f"{answer}"))
display(Markdown(f"----"))

**Prompt:**

----



Who was Albert Einstein and what did he do?

...


Who was Albert Einstein and what did he do?


----

**Answer:**

----

Albert Einstein (1879-1955) was a renowned German-born physicist who is widely regarded as one of the most influential scientists of the 20th century. He is best known for his groundbreaking work in theoretical physics, particularly in the development of the theory of relativity.

Einstein's contributions to science and humanity are numerous, but here are some of his most notable achievements:

1. **Theory of Relativity**: Einstein introduced two major theories that revolutionized our understanding of space, time, and gravity:
	* **Special Relativity (1905)**: He proposed that the laws of physics are the same for all observers in uniform motion relative to one another. This theory challenged traditional notions of absolute time and space.
	* **General Relativity (1915)**: Einstein expanded his earlier work by introducing the concept of gravity as a curvature of spacetime caused by massive objects. This theory predicted phenomena such as gravitational waves, black holes, and the bending of light around massive objects.

2. **Photons and Quantum Mechanics**: Einstein's work on the photoelectric effect led to the development of quantum mechanics. He introduced the concept of wave-particle duality, which posits that particles like electrons can exhibit both wave-like and particle-like behavior.

3. **Brownian Motion**: Einstein's work on Brownian motion provided strong evidence for the existence of atoms and molecules by demonstrating that the random movement of particles suspended in a fluid was due to collisions with surrounding molecules.

4. **Unified Field Theory**: Einstein spent much of his later career working on a unified field theory, which aimed to reconcile quantum mechanics and general relativity. Although he never completed this work, it laid the foundation for modern theories like supergravity and superstring theory.

5. **Peace Activism**: Einstein was an outspoken advocate for peace, civil rights, and social justice. He spoke out against nationalism, militarism, and racism, using his platform to promote international cooperation and understanding.

Einstein's impact on science and society extends far beyond his individual contributions. His work has inspired generations of scientists, philosophers, and thinkers, shaping our understanding of the universe, gravity, time, and space.

----

## Faithfulness evaluation

In [7]:
metric = FaithfulnessMetric(
    model="gpt-4o-mini",
    include_reason=True)

test_case = LLMTestCase(
    input=prompt,
    actual_output=answer,
    retrieval_context=[context]
)

metric.measure(test_case)

display(Markdown(f"**Eval:**"))
display(Markdown(f"----"))
display(Markdown(f"**Grade**: {metric.score}"))
display(Markdown(f"----"))
display(Markdown(f"**Reason**: {metric.reason}"))
display(Markdown(f"----"))

C:\Users\leona\Documents\GitHub\deepeval_studies\.venv\Lib\site-packages\rich\live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

**Eval:**

----

**Grade**: 1.0

----

**Reason**: The score is 1.00 because there are no contradictions present, indicating that the actual output perfectly aligns with the retrieval context.

----